In [1]:
#-*- coding : utf-8 -*-
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
from sklearn.metrics.cluster import adjusted_rand_score
from stMMR.utils import *
from stMMR.process import *
from stMMR import train_model

In [2]:
section_id = "V1_Breast_Cancer_Block_A_Section_1"
k=20

In [3]:
im_re = pd.read_csv(os.path.join('../Data',section_id,
        "image_representation/VIT_pca_representation.csv"), header=0, index_col=0, sep=',')
print(section_id, k)

V1_Breast_Cancer_Block_A_Section_1 20


In [4]:
adata = sc.read_visium("../Data/V1_Breast_Cancer_Block_A_Section_1",
                count_file="V1_Breast_Cancer_Block_A_Section_1_filtered_feature_bc_matrix.h5")
adata.var_names_make_unique()
prefilter_genes(adata, min_cells=3)  # avoiding all genes are zeros
# prefilter_specialgenes(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
adata.obsm["im_re"] = im_re

Ann_df = pd.read_csv("../Data/V1_Breast_Cancer_Block_A_Section_1/metadata.tsv", sep="	", header=0, na_filter=False,
                     index_col=0)
adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'fine_annot_type']
adata.obs['ground_truth'] = adata.obs['Ground Truth']
adata =  adata[:, adata.var['highly_variable']]

In [5]:
adata.obsm["adj"] = calculate_adj_matrix(adata)
adata= train_model.train(adata,k,n_epochs=200,h=[3000,3000],l=0.63,lr=0.0000001,radius=50)

[2024-10-01 17:05:18] Epoch: 0/200, Loss: 17.4305
[2024-10-01 17:05:54] Epoch: 5/200, Loss: 17.4265
[2024-10-01 17:06:28] Epoch: 10/200, Loss: 17.4225
[2024-10-01 17:07:01] Epoch: 15/200, Loss: 17.4184
[2024-10-01 17:07:35] Epoch: 20/200, Loss: 17.4142
[2024-10-01 17:08:09] Epoch: 25/200, Loss: 17.4104
[2024-10-01 17:08:43] Epoch: 30/200, Loss: 17.4063
[2024-10-01 17:09:16] Epoch: 35/200, Loss: 17.4028
[2024-10-01 17:09:48] Epoch: 40/200, Loss: 17.3987
[2024-10-01 17:10:22] Epoch: 45/200, Loss: 17.3945
[2024-10-01 17:10:55] Epoch: 50/200, Loss: 17.3904
[2024-10-01 17:11:28] Epoch: 55/200, Loss: 17.3865
[2024-10-01 17:12:06] Epoch: 60/200, Loss: 17.3823
[2024-10-01 17:12:46] Epoch: 65/200, Loss: 17.3787
[2024-10-01 17:13:20] Epoch: 70/200, Loss: 17.3743
[2024-10-01 17:13:55] Epoch: 75/200, Loss: 17.3702
[2024-10-01 17:14:29] Epoch: 80/200, Loss: 17.3669
[2024-10-01 17:15:02] Epoch: 85/200, Loss: 17.3624
[2024-10-01 17:15:35] Epoch: 90/200, Loss: 17.3589
[2024-10-01 17:16:08] Epoch: 95/2

In [6]:
obs_df = adata.obs.dropna()
ARI = adjusted_rand_score(obs_df['stMMR'], obs_df['Ground Truth'])
print('Adjusted rand index = %.5f' % ARI)

Adjusted rand index = 0.64748
